<a href="https://colab.research.google.com/github/surabhipandey18/Brain-Tumor-/blob/main/Presence_of_Brain_Tumor%2BXplainableAI_with_GradCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
navoneel_brain_mri_images_for_brain_tumor_detection_path = kagglehub.dataset_download('navoneel/brain-mri-images-for-brain-tumor-detection')
sartajbhuvaji_brain_tumor_classification_mri_path = kagglehub.dataset_download('sartajbhuvaji/brain-tumor-classification-mri')

print('Data source import complete.')


# **Classifying MRI images for the presence of tumor/no-tumor and show the Grad-CAM for selected images**

In [ ]:
#installing dependency
!pip install tf_keras_vis

In [ ]:
#importing dependencies
import numpy as np
import tensorflow as tf
import cv2
from tensorflow import keras
import sklearn as sk
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import os, shutil
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.utils.scores import CategoricalScore

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("navoneel/brain-mri-images-for-brain-tumor-detection")

print("Path to dataset files:", path)

In [ ]:
#cropping brain images to removing skull and focus on brain
def brain_crop(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Otsu threshold for brain extraction
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Find brain region (largest connected component)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key=cv2.contourArea)

    x, y, w, h = cv2.boundingRect(cnt)
    cropped = gray[y:y+h, x:x+w]

    # Resize to model input size
    cropped = cv2.resize(cropped, (224, 224))

    # Convert grayscale → RGB so model still sees 3 channels
    cropped = cv2.cvtColor(cropped, cv2.COLOR_GRAY2RGB)

    return cropped

In [ ]:
#installing file and classify it into tumor and non-tumor files
file = "/kaggle/input/brain-tumor-classification-mri/Training/"
binary = "/kaggle/working/binary_dataset/"

os.makedirs(binary + "tumor", exist_ok=True)
os.makedirs(binary + "no_tumor", exist_ok=True)

# Process tumor images
for cls in ["glioma_tumor", "meningioma_tumor", "pituitary_tumor"]:
    class_path = os.path.join(file, cls)
    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        img = cv2.imread(img_path)

        if img is None:
            continue

        cropped = brain_crop(img)
        cv2.imwrite(os.path.join(binary + "tumor", img_file), cropped)

# Process no tumor images
no_tumor_path = os.path.join(file, "no_tumor")
for img_file in os.listdir(no_tumor_path):
    img_path = os.path.join(no_tumor_path, img_file)
    img = cv2.imread(img_path)

    if img is None:
        continue

    cropped = brain_crop(img)
    cv2.imwrite(os.path.join(binary + "no_tumor", img_file), cropped)

print("Skull-stripped binary dataset created at:", binary)
binary_path = '/kaggle/working/binary_dataset/'

In [ ]:
#printing 1 image out of each class
plt.figure(figsize=(12, 12))
classes = os.listdir(binary)
for i in range(len(classes)):
  class_path = os.path.join(binary_path, classes[i])
  if os.path.isdir(class_path):
    images_in_class = os.listdir(class_path)
    if images_in_class:
      plt.subplot(1, 4, i+1)
      img_path = os.path.join(class_path, images_in_class[0])
      plt.imshow(image.load_img(img_path))
      plt.axis("off")
      plt.title(classes[i])
plt.tight_layout()
plt.show()

In [ ]:
#data Augmentation
datagen = ImageDataGenerator(rescale= 1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(binary, target_size = (224, 224), batch_size = 32, class_mode = 'categorical', subset='training')

validation_gen = datagen.flow_from_directory(binary, target_size = (224, 224), batch_size = 32, class_mode = 'categorical', subset='validation')

In [ ]:
#ResNet-50 model as the baseline
resnet_model = ResNet50(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
resnet_model.trainable = False
x = layers.GlobalAveragePooling2D()(resnet_model.output)
x = layers.Dropout(0.3)(x)
output = layers.Dense(2, activation='softmax')(x)

model = Model(inputs= resnet_model.input, outputs = output)
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
with tf.device('/device:GPU:0'):
  history = model.fit(train_gen, validation_data= validation_gen, batch_size = 32, epochs= 5, verbose= 1)

In [ ]:
#unfreezig last 20 layers of ResNet-50 model keeping all 'Batchnormalization' layers freezed, finetuning ResNet model
model.trainable = True
for layer in model.layers[:-20]:
  if "BatchNormalization" in layer.__class__.__name__:
    layer.trainable = False

model.compile(optimizer = Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics = ['accuracy'])

with tf.device("/device:GPU:0"):
    history_fine_tuned = model.fit(train_gen, validation_data= validation_gen, batch_size = 32, epochs = 20, verbose= 1)

**Grad-CAM, or Gradient-weighted Class Activation Mapping, is an explainable AI technique that produces a heatmap to show which parts of an image a convolutional neural network (CNN) focuses on to make a specific prediction. It uses the gradients of the classification score to highlight the most important pixels or regions in an image, helping us understand, debug, and improve the model's decision-making process.
So now we will be displaying the Grad-CAM for the unfreezed model and take the last layer, which is "conv5_block3_out" and see how that layer is predicting.**

In [ ]:
#gradcam for a few images
def gradcam(model, img_path, layer_name):
    grad_model = tf.keras.Model([model.inputs], [model.get_layer(layer_name).output, model.output])

    img = cv2.imread(img_path)
    if img is None:
        raise ValueError("Image path invalid: " + img_path)

    if len(img.shape) == 2 or img.shape[-1] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)

    with tf.GradientTape() as tape:
        img_array = tf.convert_to_tensor(img, dtype=tf.float32)
        conv_outputs, predictions = grad_model([img_array])
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis= (0,1,2))

    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis= -1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= tf.reduce_max(heatmap)
    heatmap = heatmap.numpy()
    return heatmap



In [ ]:
#function for displaying Grad-CAM for images
def display_grad(img_path, heatmap, alpha= 0.6):
    img= cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)  # scale 0–1 → 0–255
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)

    plt.figure(figsize=(8,6))
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

In [ ]:
#defining parameters for displaying Grad-CAM
img_path = '/kaggle/input/brain-tumor-classification-mri/Testing/glioma_tumor/image(1).jpg'
heatmap = gradcam(model, img_path, "conv5_block3_out")
display_grad(img_path, heatmap)

**The fact that our Grad-CAM is not showing good results and the fact that the accuracy of the model is also low could be due to the issue that the images are not of the same MRI sequence. We can try this Grad-CAM, we need modality consistency.**